In [1]:
import re

In [3]:
def contains(a,b):
    a = list(a)
    b = list(b)
    result = True
    for element in a:
        result = result and (element in b)
    return result

In [107]:
def replace_negated(sentence):
    #remove negations from sentence with values placed
    not_replacement = {"~0" : "1", "~1" : "0"}
    for val in not_replacement.keys():
        not_parts = re.search('[\~\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
        while not_parts != None:
            not_part = not_parts.group(0)
            not_part = not_part[1:-1]
            sentence = sentence.replace(not_part, not_replacement[val])
            sentence = simplify(sentence)
            not_parts = re.search('[\(\&\|\-\>]' + val + '[\)\&\|\-\<]', sentence)
    if '~' in sentence:
        sentence = replace_negated(sentence)
    return sentence

In [99]:
def simplify(sentence):
    parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    while parts != None:
        sentence = sentence.replace(parts.group(0), parts.group(0)[1:-1])
        parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    return sentence

In [194]:
def splitted(sentence, op):
    #check if sentence is of form (P op Q), where P and Q are valid formulas and op is given operation
    if sentence[0] == '(':
        sentence = sentence[1:-1]
    counter = 0
    for idx in range(len(sentence)):
        if sentence[idx] == '(':
            counter += 1
        else:
            if sentence[idx] == ')':
                counter -= 1
            else:
                if (counter == 0) and idx + len(op) <= len(sentence):
                    flag = True
                    for op_idx in range(len(op)):
                        flag = flag and sentence[op_idx + idx] == op[op_idx]
                    if flag:
                        return [construct_sentence(sentence[:idx]), construct_sentence(sentence[idx+len(op):])]
    return None

In [184]:
def construct_sentence(s):
    if s[0] != '(':
        return '(' + s + ')'
    else:
        return s

In [102]:
def evaluate(sentence, model):
    keys = model.keys()
    
    #put variables values
    for key in keys:
        parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
        while (parts != None):
            part = parts.group(0)
            part = part[1:-1]
            sentence = sentence.replace(part, model[key])
            parts = re.search('[~\(\&\|\-\>](' + key + ')[\)\&\|\-\<]', sentence)
            
    sentence = simplify(sentence)
    sentence = replace_negated(sentence)
    
    #compute sentence value
    parts = re.search('(\([^\(\)]*\))', sentence)
    while parts != None:
        part = parts.group(0)
        sentence = sentence.replace(part, evaluate_simple(part))
        sentence = replace_negated(sentence)
        sentence = simplify(sentence)
        parts = re.search('(\([^\(\)]*\))', sentence)
    return bool(int(sentence))

In [5]:
def evaluate_simple(sentence):
    if sentence[0] == '(' and sentence[-1] == ')':
        sentence = sentence[1:-1]
    
    #eval and = &
    if sentence == '0&0' or sentence== '0&1' or sentence == '1&0':
        return '0'
    else:
        if sentence == '1&1':
            return '1'
    
    #eval or = |
    if sentence == '1|1' or sentence== '0|1' or sentence == '1|0':
        return '1'
    else:
        if sentence == '0|0':
            return '0'
    
    #eval implication = ->
    if sentence == '0->0' or sentence== '0->1' or sentence == '1->1':
        return '1'
    else:
        if sentence == '1->0':
            return '0'
    
    #eval equivalence = <->
    if sentence == '0<->0' or sentence == '1<->1':
        return '1'
    else:
        if sentence == '1<->0' or sentence == '0<->1':
            return '0'
    
    return sentence

In [6]:
def evaluate_simple_test():
    val = ["0", "1"]
    operators = ['&', '|', '->', '<-', '<->']
    for op in operators:
        for v1 in val:
            for v2 in val:
                sentence = '(' + v1 + op + v2 + ')'
                res = evaluate_simple(sentence)
                print sentence + "   " + res
                sentence = v1 + op + v2
                res = evaluate_simple(sentence)
                print sentence + "   " + res

In [19]:
def extract_symbols(sentence):
    result = []
    symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    while (symbols != None):
        symbol = symbols.group(0)
        symbol = symbol[1:-1]
        result.append(symbol)
        sentence = sentence.replace(symbol, '')
        symbols = re.search('[\(\&\|\>\-\~][^\(\)\&\|\<\>\-\~]+[\)\&\|\<\-]', sentence)
    return result

In [20]:
def tt_entails(KB, sentence):
    #input: KB is list of sentences(str)
    #       sentence is a str
    
    #find all symbols
    symbols = extract_symbols(sentence)
    for s in KB:
        symbols += extract_symbols(s)
    symbols = list(set(symbols))
    
    return tt_check_all(KB, sentence, symbols, {})

In [21]:
def satisfied(KB, model):
    if type(KB) == list:
        return satisfied_kb(KB, model)
    if type(KB) == str:
        return evaluate(KB, model)

def satisfied_kb(KB, model):
    symbols = []
    for s in KB:
        symbols += extract_symbols(s)
    symbols = set(symbols)
    
    if contains(symbols, set(model.keys())):
        result = True
        for s in KB:
            result = result and evaluate(s, model)
        return result
    else:
        return False

In [22]:
def tt_check_all(KB, sentence, symbols, model):
    if len(symbols) == 0:
        if satisfied(KB, model):
            return satisfied(sentence, model)
        else:
            return True
    else:
        result = True
        model[symbols[0]] = '0'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        model[symbols[0]] = '1'
        result = result and tt_check_all(KB, sentence, symbols[1:], model)
        return result

In [72]:
string = '(((~(A->B)|C)->D)&F)'

In [74]:
evaluate(string,model)

True

In [94]:
model = {"A" : "0", "B" : "1", "C" : "0", "D" : "0", "F" : "1"}

In [57]:
KB = ['(A|B)', '(B&F)', '(~C->F)']

In [58]:
satisfied(KB,model)

True

In [34]:
KB1 = ['(P)', '(P->Q)']
sentence1 = '(Q)'

In [35]:
tt_entails(KB1, sentence1)

True

In [214]:
def cnf_to_clauses(cnf):
    #given string with cnf conver it to list of clauses
    #note: the resulting clauses have less bracketing then it is required by convert_to_cnf as we allow A|B|C|D|E etc.
    if cnf[0] == '(':
        cnf = cnf[1:-1]
    cnf = cnf.split('&')
    result = []
    for clause in cnf:
        clause = re.sub('\(|\)', '', clause)
        result.append('(' + clause + ')')
    return result

In [215]:
def convert_to_cnf(sentence):
    #some changes to form of sentence:
    #     replace (v) with v, where v is valiable
    #     add outer brackets if we do not have them
    parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    while parts != None:
        sentence = sentence.replace(parts.group(0), parts.group(0)[1:-1])
        parts = re.search('\([^\~\<\>\(\)\-\&\|]*\)', sentence)
    if sentence[0] != '(':
        sentence = '(' + sentence + ')'
    sentence = simplify(sentence)
    
    print sentence
    
    #case 1: sentence is just variable:
    parts = re.search('^\([^\~\(\)\<\-\>\&\|]*\)$', sentence)
    if parts != None:
        return sentence
    
    #case 2: sentence is (V1&V2), where Vi are variables
    parts = re.search('^\((\~?[^\(\)\<\>\-\&\|]*)\&(\~?[^\(\)\<\>\-\&\|]*)\)$', sentence)
    if parts != None:
        return '((' + parts.group(1) + ')&(' + parts.group(2) + '))'
    
    #case 3: sentence is (V1&V2), where Vi are formulas
    #parts = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', sentence)
    split = splitted(sentence, '&')
    if split != None:
        return '(' + construct_sentence(convert_to_cnf(split[0])) + '&' + construct_sentence(convert_to_cnf(split[1])) + ')'
    
    #case 4: sentence is (V1|V2), where Vi are variables
    parts = re.search('^\((\~?[^\(\)\<\>\-\&\|]*)\|(\~?[^\(\)\<\>\-\&\|]*)\)$', sentence)
    if parts != None:
        return sentence
    
    #case 5: sentence is (V1|V2), where Vi are formulas
    #parts = re.search('^\((\~?\(.*\))\|(\~?\(.*\))\)$', sentence)
    split = splitted(sentence, '|')
    if split != None:
        #find cnfs for both sides
        cnf_l = construct_sentence(convert_to_cnf(split[0]))
        cnf_r = construct_sentence(convert_to_cnf(split[1]))
        #splits cnf into lists of clauses
        cnf_l = cnf_to_clauses(cnf_l)
        cnf_r = cnf_to_clauses(cnf_r)
        result = '('
        for left_particle in cnf_l:
            for right_particle in cnf_r:
                result = result + '(' + construct_sentence(left_particle) + '|' + construct_sentence(right_particle) + ')&' 
        result = result[:-1]
        result += ')'
        return result
    
    #case 6: sentence is negated: (~V), where V is a varible
    parts = re.search('^\(\~[^\~\<\>\(\)\-\&\|]*\)$', sentence)
    if parts != None:
        return sentence
    
    #case 7: sentence is doubly negated: (~(~V)) or (~(~(V))) or (~~V) or (~~(V)), where V is a formula or variable
    parts = re.search('^\(\~\~(.*)\)$', sentence)
    if parts != None:
        return convert_to_cnf(parts.group(1))
    parts = re.search('^\(\~\(\~(.*)\)\)$', sentence)
    if parts != None:
        return convert_to_cnf(parts.group(1))
    
    #case 8: sentence is of form (~(P&Q)), where P and Q are formulas or variables
    parts = re.search('^\(\~(\(.*\&.*\))\)$', sentence)
    if parts != None:
        split = splitted(parts.group(1),'&')
        if split != None:
            return convert_to_cnf('(~' + split[0] + '|~' + split[1] + ')')
    
    #case 9: sentence is of form (~(P|Q)), where P and Q are formulas or variables
    parts = re.search('^\(\~(\(.*\|.*\))\)$', sentence)
    if parts != None:
        split = splitted(parts.group(1),'|')
        if split != None:
            return convert_to_cnf('(~' + split[0] + '&~' + split[1] + ')')
    
    #case 10: sentence is of form (P->Q), where P and Q are formulas or variables
    split = splitted(sentence, '->')
    if split != None:
        return convert_to_cnf('(~' + split[0] + '|' + split[1] + ')')
    
    #case 11: sentence is of form (P<->Q), where P and Q are formulas or variables
    split = splitted(sentence, '<->')
    if split != None:
        return convert_to_cnf('((' + split[0] + '&' + split[1] + ')|(~' + split[0] + '&' + split[1] + '))')
    
    return sentence

In [70]:
t = re.search('^\((\(.*\))\&(\(.*\))\)$', '((A&B)&(C&D))')

In [71]:
t.group(1)

'(A&B)'

In [187]:
convert_to_cnf('(~(A&B)&(A&B))')

(~(A&B)&(A&B))
(~(A&B))
(~A|~B)
(A&B)


'((~A|~B)&((A)&(B)))'

In [209]:
convert_to_cnf('(~A&~B)')

(~A&~B)


'((~A)&(~B))'

In [218]:
simplify(convert_to_cnf('(((P1&P2)&P3)|Q1)'))

(((P1&P2)&P3)|Q1)
((P1&P2)&P3)
(P1&P2)
P3
Q1


'((P1|Q1)&(P2|Q1)&(P3|Q1))'

In [212]:
convert_to_cnf('((P1&P2)&P3)')

((P1&P2)&P3)
(P1&P2)
P3


'(((P1)&(P2))&(P3))'

In [ ]:
((((P1)|(Q1))&(((P1)|(Q2))&((P2))|(Q1))&((P2))|(Q2))&((P3)|(Q1))&((P3)|(Q2)))

In [197]:
convert_to_cnf('((A->B)&(B|C))')

((A->B)&(B|C))
(A->B)
(~A|B)
(B|C)


'((~A|B)&(B|C))'

In [196]:
convert_to_cnf('(A->B)')

(A->B)
(~A|B)


'(~A|B)'

In [82]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B)&(A&B))')

In [89]:
convert_to_cnf(t.group(1))

In [92]:
'((A|B)&(C|D))'[1:-1].split('&')

['(A|B)', '(C|D)']

In [109]:
evaluate('(~~F)', model)

True

In [95]:
model

{'A': '0', 'B': '1', 'C': '0', 'D': '0', 'F': '1'}

In [108]:
replace_negated('(~~1)')

'1'

In [100]:
simplify('(~(0))')

'(~0)'

In [110]:
s = '(~(~((A->B)&(C->D))))'

In [111]:
parts = re.search('^\(\~\(?\~(.*)\)$', s)

In [112]:
parts.group(1)

'((A->B)&(C->D)))'

In [127]:
convert_to_cnf('(~(~A&B))')

(~(~A&B))
(A&B)


'((A)&(B))'

In [118]:
parts = re.search('^\(\~\~(.*)\)$', '(~~A)')

In [119]:
parts.group(1)

'A'

In [120]:
convert_to_cnf('A')

'(A)'

In [141]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B)&(A&B))')

In [142]:
t == None

False

In [145]:
t.group(2)

'(A&B)'

In [159]:
t = re.search('^\((\~?\(.*\))\&(\~?\(.*\))\)$', '(~(A&B))')

In [161]:
t